# Kapitola 20: Klasifikace obrazku - Prvni kroky v pocitacovem videni

## Naucte AI rozpoznavat obrazky pomoci datasetu CIFAR-10

---

### Co se naucite:
- Nacitat a prozkoumat obrazovy dataset CIFAR-10
- Pripravit data pro strojove uceni (normalizace, zplosteni)
- Trenovat klasifikator obrazku pomoci Logisticke regrese
- Vyhodnocovat presnost modelu a analyzovat chyby
- Vizualizovat predikce a confusion matrix

### Motivacni priklad:
Predstavte si, ze jste digitalni biolog s krabici 50 000 fotek. Jsou na nich zaby, kone,
lode, letadla... Vas ukol je vytvorit system, ktery je automaticky roztridi.
Dnes presne takovy system postavime!

---

## 1. Instalace a import knihoven

In [ ]:
# Instalace knihoven pro Google Colab
!pip install tensorflow scikit-learn matplotlib numpy -q

print("Knihovny uspesne nainstalovany!")

In [ ]:
# Import knihoven
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# TensorFlow/Keras pro nacteni datasetu
from tensorflow.keras.datasets import cifar10

# Scikit-learn pro klasifikaci
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Nastaveni vizualizaci
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Vsechny knihovny importovany!")
print("Tema: Klasifikace obrazku - pocitacove videni")

## 2. Dataset CIFAR-10

**CIFAR-10** je slavny benchmark dataset pro pocitacove videni:
- 60,000 barevnych obrazku (32x32 pixelu)
- 10 kategorii: letadlo, auto, ptak, kocka, jelen, pes, zaba, kun, lod, kamion
- 50,000 trenovacich + 10,000 testovacich obrazku

In [ ]:
# Nacteni datasetu CIFAR-10
print("=" * 60)
print("NACTENI DATASETU CIFAR-10")
print("=" * 60)

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Nazvy kategorii
kategorie = ['letadlo', 'auto', 'ptak', 'kocka', 'jelen', 
             'pes', 'zaba', 'kun', 'lod', 'kamion']

print(f"\nTrenovaci data:")
print(f"  Obrazky: {x_train.shape}")
print(f"  Labely: {y_train.shape}")

print(f"\nTestovaci data:")
print(f"  Obrazky: {x_test.shape}")
print(f"  Labely: {y_test.shape}")

print(f"\nKategorie ({len(kategorie)}): {kategorie}")
print(f"\nRozmer jednoho obrazku: {x_train[0].shape}")
print(f"  = 32x32 pixelu, 3 barevne kanaly (RGB)")

In [ ]:
# Vizualizace ukazkovych obrazku
plt.figure(figsize=(15, 6))

# Zobrazime 2 obrazky z kazde kategorie
for i, kat in enumerate(kategorie):
    # Najdeme obrazky dane kategorie
    indices = np.where(y_train.flatten() == i)[0]
    
    plt.subplot(2, 5, i + 1)
    plt.imshow(x_train[indices[0]])
    plt.title(kat, fontsize=11)
    plt.axis('off')

plt.suptitle('Ukazkove obrazky z datasetu CIFAR-10', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nKazdy obrazek ma rozmer 32x32 pixelu - jsou male, ale AI se z nich nauci!")

In [ ]:
# Distribuce kategorii v datasetu
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Trenovaci data
train_counts = np.bincount(y_train.flatten())
ax1.bar(kategorie, train_counts, color='steelblue', edgecolor='black')
ax1.set_xlabel('Kategorie')
ax1.set_ylabel('Pocet obrazku')
ax1.set_title('Rozdeleni trenovacich dat')
ax1.tick_params(axis='x', rotation=45)

# Testovaci data
test_counts = np.bincount(y_test.flatten())
ax2.bar(kategorie, test_counts, color='coral', edgecolor='black')
ax2.set_xlabel('Kategorie')
ax2.set_ylabel('Pocet obrazku')
ax2.set_title('Rozdeleni testovacich dat')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"Dataset je vyvazeny - kazda kategorie ma {train_counts[0]} trenovacich obrazku.")

## 3. Priprava dat pro strojove uceni

Pred trenovanim modelu musime data pripravit:

1. **Normalizace** - prevod hodnot pixelu z 0-255 na 0-1
2. **Zplosteni (Flattening)** - prevod 2D obrazku na 1D vektor

In [ ]:
# Demonstrace jak pocitac "vidi" obrazek
print("=" * 60)
print("JAK POCITAC VIDI OBRAZEK")
print("=" * 60)

# Vybereme prvni obrazek
priklad = x_train[0]

print(f"\nRozmer obrazku: {priklad.shape}")
print(f"  32 radku x 32 sloupcu x 3 barevne kanaly (RGB)")
print(f"\nHodnoty pixelu (prvni radek, cerveny kanal):")
print(priklad[0, :10, 0])  # Prvnich 10 pixelu
print(f"\nRozsah hodnot: {priklad.min()} - {priklad.max()}")

# Vizualizace jednotlivych kanalu
fig, axes = plt.subplots(1, 4, figsize=(14, 3))

axes[0].imshow(priklad)
axes[0].set_title('Originalni obrazek')
axes[0].axis('off')

axes[1].imshow(priklad[:,:,0], cmap='Reds')
axes[1].set_title('Cerveny kanal (R)')
axes[1].axis('off')

axes[2].imshow(priklad[:,:,1], cmap='Greens')
axes[2].set_title('Zeleny kanal (G)')
axes[2].axis('off')

axes[3].imshow(priklad[:,:,2], cmap='Blues')
axes[3].set_title('Modry kanal (B)')
axes[3].axis('off')

plt.suptitle(f'Kategorie: {kategorie[y_train[0][0]]}', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Normalizace dat
print("=" * 60)
print("NORMALIZACE DAT")
print("=" * 60)

# Prevod na float a normalizace na rozsah 0-1
x_train_norm = x_train.astype('float32') / 255.0
x_test_norm = x_test.astype('float32') / 255.0

print(f"\nPred normalizaci:")
print(f"  Rozsah hodnot: {x_train.min()} - {x_train.max()}")
print(f"  Datovy typ: {x_train.dtype}")

print(f"\nPo normalizaci:")
print(f"  Rozsah hodnot: {x_train_norm.min():.2f} - {x_train_norm.max():.2f}")
print(f"  Datovy typ: {x_train_norm.dtype}")

print("\nProc normalizovat?")
print("  - Model se uci rychleji")
print("  - Lepsi numericka stabilita")
print("  - Vsechny features maji stejnou vahu")

In [ ]:
# Zplosteni obrazku
print("=" * 60)
print("ZPLOSTENI (FLATTENING) OBRAZKU")
print("=" * 60)

# Zplosteni: (N, 32, 32, 3) -> (N, 3072)
n_train = x_train_norm.shape[0]
n_test = x_test_norm.shape[0]

x_train_flat = x_train_norm.reshape(n_train, -1)  # -1 = automaticky dopocita
x_test_flat = x_test_norm.reshape(n_test, -1)

# Zplosteni labelu
y_train_flat = y_train.flatten()
y_test_flat = y_test.flatten()

print(f"\nPred zplostenim:")
print(f"  Tvar: {x_train_norm.shape}")
print(f"  = 50000 obrazku, kazdy 32x32x3")

print(f"\nPo zplosteni:")
print(f"  Tvar: {x_train_flat.shape}")
print(f"  = 50000 vzorku, kazdy ma {32*32*3} features")

print("\nProc zplostit?")
print("  Jednoduche modely (Logisticka regrese, KNN) ocekavaji 1D vstup.")
print("  Pokrocilejsi modely (CNN) pracuji primo s 2D obrazky.")

## 4. Trenovani klasifikatoru

Pouzijeme **Logistickou regresi** - jednoduchy, ale efektivni klasifikator.

**Poznamka:** Pro urychleni pouzijeme jen cast dat (10,000 obrazku).

In [ ]:
# Vyber podmnoziny dat pro rychlejsi trenovani
print("=" * 60)
print("PRIPRAVA TRENOVACICH DAT")
print("=" * 60)

# Pouzijeme 10,000 obrazku pro trenovani (pro rychlost v Colab)
n_samples = 10000

X_subset = x_train_flat[:n_samples]
y_subset = y_train_flat[:n_samples]

print(f"\nTrenovaci data:")
print(f"  Pocet vzorku: {X_subset.shape[0]}")
print(f"  Pocet features: {X_subset.shape[1]}")

print(f"\nTestovaci data:")
print(f"  Pocet vzorku: {x_test_flat.shape[0]}")

In [ ]:
# Trenovani Logisticke regrese
print("=" * 60)
print("TRENOVANI LOGISTICKE REGRESE")
print("=" * 60)

print("\nTrenuji model... (muze trvat 1-2 minuty)")

# Vytvoreni a trenovani modelu
model_lr = LogisticRegression(
    max_iter=100,           # Max pocet iteraci
    solver='lbfgs',         # Optimalizacni algoritmus
    multi_class='multinomial',  # Vicetridni klasifikace
    n_jobs=-1,              # Paralelni vypocet
    verbose=1               # Zobrazeni prubeznych informaci
)

model_lr.fit(X_subset, y_subset)

print("\nModel uspesne natrenovan!")

## 5. Evaluace modelu

In [ ]:
# Predikce na testovacich datech
print("=" * 60)
print("EVALUACE MODELU")
print("=" * 60)

# Predikce
y_pred = model_lr.predict(x_test_flat)

# Presnost
presnost = accuracy_score(y_test_flat, y_pred)

print(f"\nCelkova presnost: {presnost*100:.2f}%")
print(f"\n(Nahodne hadani by melo presnost 10% - 10 kategorii)")
print(f"Nas model je {presnost/0.1:.1f}x lepsi nez nahodne hadani!")

In [ ]:
# Detailni report
print("\n" + "=" * 60)
print("DETAILNI CLASSIFICATION REPORT")
print("=" * 60)

print(classification_report(y_test_flat, y_pred, target_names=kategorie))

In [ ]:
# Confusion Matrix
plt.figure(figsize=(12, 10))

cm = confusion_matrix(y_test_flat, y_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=kategorie, yticklabels=kategorie)
plt.xlabel('Predikovana kategorie')
plt.ylabel('Skutecna kategorie')
plt.title('Confusion Matrix (normalizovana) - Logisticka regrese', fontsize=14)

plt.tight_layout()
plt.show()

print("Interpretace:")
print("  - Diagonala = spravne klasifikovane (vyssi = lepsi)")
print("  - Mimo diagonalu = chyby (model si plete tyto kategorie)")
print("  - Napr. kocka/pes - casto se pletou (podobne tvary)")

In [ ]:
# Presnost pro kazdu kategorii
presnosti_kategorie = cm.diagonal() / cm.sum(axis=1)

plt.figure(figsize=(12, 5))

colors = plt.cm.RdYlGn(presnosti_kategorie)
bars = plt.bar(kategorie, presnosti_kategorie * 100, color=colors, edgecolor='black')

plt.axhline(y=presnost*100, color='red', linestyle='--', label=f'Prumerna presnost ({presnost*100:.1f}%)')
plt.xlabel('Kategorie')
plt.ylabel('Presnost (%)')
plt.title('Presnost klasifikace pro jednotlive kategorie', fontsize=14)
plt.legend()
plt.xticks(rotation=45)

# Popisky
for bar, p in zip(bars, presnosti_kategorie):
    plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1,
             f'{p*100:.1f}%', ha='center', fontsize=9)

plt.ylim(0, 60)
plt.tight_layout()
plt.show()

# Nejlepsi a nejhorsi kategorie
nejlepsi = kategorie[np.argmax(presnosti_kategorie)]
nejhorsi = kategorie[np.argmin(presnosti_kategorie)]
print(f"\nNejlepe rozpoznatelna kategorie: {nejlepsi} ({presnosti_kategorie.max()*100:.1f}%)")
print(f"Nejobtiznejsi kategorie: {nejhorsi} ({presnosti_kategorie.min()*100:.1f}%)")

## 6. Vizualizace predikci

In [ ]:
# Vizualizace predikcí na testovacich datech
print("=" * 60)
print("UKAZKA PREDIKCI")
print("=" * 60)

plt.figure(figsize=(15, 8))

# Vybereme 15 nahodnych obrazku
np.random.seed(42)
indices = np.random.choice(len(x_test), 15, replace=False)

for i, idx in enumerate(indices):
    plt.subplot(3, 5, i + 1)
    plt.imshow(x_test[idx])
    
    skutecna = kategorie[y_test_flat[idx]]
    predikovana = kategorie[y_pred[idx]]
    
    # Zelena pokud spravne, cervena pokud spatne
    if skutecna == predikovana:
        barva = 'green'
        plt.title(f'{predikovana}', color=barva, fontsize=10)
    else:
        barva = 'red'
        plt.title(f'{predikovana}\n(spravne: {skutecna})', color=barva, fontsize=9)
    
    plt.axis('off')

plt.suptitle('Predikce na testovacich datech (zelena=spravne, cervena=chyba)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Analyza chyb - nejcastejsi zameny
print("\n" + "=" * 60)
print("ANALYZA CHYB - NEJCASTEJSI ZAMENY")
print("=" * 60)

# Najdeme nejcastejsi zameny (mimo diagonalu)
cm_errors = cm.copy()
np.fill_diagonal(cm_errors, 0)  # Vynulujeme diagonalu

# Top 5 nejcastejsich zamen
top_errors = []
for _ in range(5):
    i, j = np.unravel_index(cm_errors.argmax(), cm_errors.shape)
    count = cm_errors[i, j]
    top_errors.append((kategorie[i], kategorie[j], count))
    cm_errors[i, j] = 0

print("\nTop 5 nejcastejsich zamen:")
for skutecna, predikovana, pocet in top_errors:
    print(f"  {skutecna} -> {predikovana}: {pocet} chyb")

## 7. Srovnani s KNN klasifikatorem

In [ ]:
# Srovnani s K-Nearest Neighbors
print("=" * 60)
print("SROVNANI: Logisticka regrese vs KNN")
print("=" * 60)

print("\nTrenuji KNN klasifikator... (muze trvat 1-2 minuty)")

# KNN klasifikator
model_knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
model_knn.fit(X_subset, y_subset)

# Predikce (pouzijeme mensi vzorek pro rychlost)
y_pred_knn = model_knn.predict(x_test_flat[:2000])
presnost_knn = accuracy_score(y_test_flat[:2000], y_pred_knn)

print(f"\nVysledky:")
print(f"  Logisticka regrese: {presnost*100:.2f}%")
print(f"  K-Nearest Neighbors: {presnost_knn*100:.2f}%")

In [ ]:
# Vizualizace srovnani
plt.figure(figsize=(10, 6))

modely = ['Nahodne hadani', 'KNN (k=5)', 'Logisticka regrese', 'CNN (referencni)']
presnosti = [10, presnost_knn*100, presnost*100, 90]  # CNN jako referencni hodnota
colors = ['gray', '#3498db', '#2ecc71', '#e74c3c']

bars = plt.bar(modely, presnosti, color=colors, edgecolor='black')

plt.ylabel('Presnost (%)')
plt.title('Srovnani presnosti ruznych modelu na CIFAR-10', fontsize=14)
plt.ylim(0, 100)

for bar, p in zip(bars, presnosti):
    plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1,
             f'{p:.1f}%', ha='center', fontsize=11, fontweight='bold')

# Anotace
plt.annotate('Pro vyssi presnost\npouzijte CNN!', xy=(3, 90), xytext=(2.5, 70),
            fontsize=10, ha='center',
            arrowprops=dict(arrowstyle='->', color='gray'))

plt.tight_layout()
plt.show()

print("\nZaver:")
print("  Jednoduche modely dosahnou ~30-40% presnosti.")
print("  Konvolucni neuronove site (CNN) dosahnou 90%+ presnosti!")

## 8. Mini-projekt: Vlastni klasifikator obrazku

In [ ]:
class ObrazkovyKlasifikator:
    """
    Jednoduchy klasifikator obrazku s podporou ruznych algoritmu.
    """
    
    def __init__(self, algoritmus='logistic'):
        """
        Inicializace klasifikatoru.
        
        Parametry:
        - algoritmus: 'logistic' nebo 'knn'
        """
        self.algoritmus = algoritmus
        self.kategorie = None
        self.model = None
        self.je_natrenovany = False
        
        if algoritmus == 'knn':
            self.model = KNeighborsClassifier(n_neighbors=5)
        else:
            self.model = LogisticRegression(max_iter=100, solver='lbfgs', 
                                           multi_class='multinomial')
    
    def priprav_data(self, obrazky):
        """Normalizuje a zplosti obrazky."""
        # Normalizace
        obrazky_norm = obrazky.astype('float32') / 255.0
        # Zplosteni
        n = obrazky_norm.shape[0]
        return obrazky_norm.reshape(n, -1)
    
    def trenuj(self, obrazky, labely, nazvy_kategorii):
        """Natrenuje klasifikator."""
        self.kategorie = nazvy_kategorii
        
        # Priprava dat
        X = self.priprav_data(obrazky)
        y = labely.flatten() if labely.ndim > 1 else labely
        
        print(f"Trenuji {self.algoritmus} klasifikator...")
        self.model.fit(X, y)
        self.je_natrenovany = True
        print("Trenovani dokonceno!")
    
    def predikuj(self, obrazky):
        """Predikuje kategorie pro obrazky."""
        if not self.je_natrenovany:
            raise Exception("Klasifikator neni natrenovany!")
        
        X = self.priprav_data(obrazky)
        predikce = self.model.predict(X)
        return predikce
    
    def vyhodnot(self, obrazky, labely):
        """Vyhodnoti presnost na testovacich datech."""
        predikce = self.predikuj(obrazky)
        y = labely.flatten() if labely.ndim > 1 else labely
        
        presnost = accuracy_score(y, predikce)
        return presnost
    
    def zobraz_predikce(self, obrazky, labely, n=10):
        """Vizualizuje predikce."""
        predikce = self.predikuj(obrazky[:n])
        y = labely.flatten()[:n] if labely.ndim > 1 else labely[:n]
        
        plt.figure(figsize=(15, 3))
        for i in range(n):
            plt.subplot(1, n, i + 1)
            plt.imshow(obrazky[i])
            skutecna = self.kategorie[y[i]]
            predikovana = self.kategorie[predikce[i]]
            barva = 'green' if skutecna == predikovana else 'red'
            plt.title(f'{predikovana}', color=barva, fontsize=9)
            plt.axis('off')
        plt.tight_layout()
        plt.show()


print("Trida ObrazkovyKlasifikator pripravena!")

In [ ]:
# Demonstrace pouziti tridy
print("=" * 60)
print("DEMONSTRACE: ObrazkovyKlasifikator")
print("=" * 60)

# Vytvoreni a trenovani klasifikatoru
klasifikator = ObrazkovyKlasifikator(algoritmus='logistic')
klasifikator.trenuj(x_train[:5000], y_train[:5000], kategorie)

# Vyhodnoceni
presnost = klasifikator.vyhodnot(x_test[:1000], y_test[:1000])
print(f"\nPresnost na testovacich datech: {presnost*100:.1f}%")

# Vizualizace predikci
print("\nUkazka predikci:")
klasifikator.zobraz_predikce(x_test, y_test, n=10)

## 9. Shrnutí kapitoly

### Co jsme se naučili:

1. **Dataset CIFAR-10** - 60,000 obrazku v 10 kategoriich

2. **Priprava dat**:
   - Normalizace: 0-255 -> 0-1
   - Zplosteni: 32x32x3 -> 3072

3. **Klasifikace obrazku** pomoci Logisticke regrese a KNN

4. **Evaluace** - presnost, confusion matrix, classification report

5. **Omezeni jednoduchych modelu** - ~30-40% presnost na CIFAR-10

### Cesta k vyssi presnosti:
- **Konvolucni neuronove site (CNN)** dosahnou 90%+ presnosti
- CNN zachovavaji prostorovou strukturu obrazku
- Budeme je probírat v dalsich kapitolach!

In [ ]:
# Finalni shrnutí
print("=" * 60)
print("SHRNUTÍ KAPITOLY 20")
print("=" * 60)

print("""
Dnes jsme prosli kompletnim procesem klasifikace obrazku:

1. NACTENI DAT
   - Dataset CIFAR-10 (50,000 trenovacich + 10,000 testovacich)
   - 10 kategorii: letadla, auta, zvířata, lode...

2. PRIPRAVA DAT
   - Normalizace: pixely 0-255 -> 0-1
   - Zplosteni: 32x32x3 -> 3072 features

3. TRENOVANI MODELU
   - Logisticka regrese
   - K-Nearest Neighbors

4. EVALUACE
   - Presnost ~30-40% (vs 10% nahodne hadani)
   - Confusion matrix - vizualizace chyb

5. DALSI KROKY
   - CNN pro 90%+ presnost
   - Transfer learning
   - Data augmentation

Gratulujeme! Prave jste dokoncili Blok 2 kurzu!
""")

## 10. Cviceni pro procviceni

### Cviceni 1:
Zkuste zmenit pocet trenovacich vzorku (5000, 20000, 50000) a sledujte, jak se meni presnost.

### Cviceni 2:
Experimentujte s parametrem `n_neighbors` u KNN (1, 3, 5, 10, 20).

### Cviceni 3:
Zkuste klasifikovat pouze 2 kategorie (napr. kocky vs psy). Zlepsí se presnost?

In [ ]:
# Prostor pro cviceni
print("Cviceni 3: Binární klasifikace kocka vs pes")
print("-" * 40)

# Vyber pouze kocky (3) a psy (5)
idx_train = np.where((y_train_flat == 3) | (y_train_flat == 5))[0]
idx_test = np.where((y_test_flat == 3) | (y_test_flat == 5))[0]

X_binary_train = x_train_flat[idx_train]
y_binary_train = y_train_flat[idx_train]
X_binary_test = x_test_flat[idx_test]
y_binary_test = y_test_flat[idx_test]

print(f"Trenovaci vzorky (kocka+pes): {len(X_binary_train)}")
print(f"Testovaci vzorky: {len(X_binary_test)}")

# Trenink a evaluace
model_binary = LogisticRegression(max_iter=100)
model_binary.fit(X_binary_train[:5000], y_binary_train[:5000])

presnost_binary = model_binary.score(X_binary_test, y_binary_test)
print(f"\nPresnost (kocka vs pes): {presnost_binary*100:.1f}%")
print(f"Zlepseni oproti 10-tridni klasifikaci!")

---

## Dalsi kroky

V dalsim bloku se podivame na **neuronove site** a **hluboke uceni (Deep Learning)** -
technologie, ktere dosahují state-of-the-art vysledku v pocitacovem videni.

---

*Kapitola 20 - Klasifikace obrazku | Kurz AI pro zacatecniky*

**KONEC BLOKU 2**